In [ ]:
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed, Dropout
from keras.layers import LSTM
from keras.utils.np_utils import to_categorical
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import os

# Define handy functions 

In [1]:
def standard_scale(*args):
    """
    Standardize features by removing the mean and scaling to unit variance
    :param args: One or multidimensional numpy array.
    :return: The scaler-object(s)
    """
    from sklearn.preprocessing import StandardScaler

    for idx, arg in enumerate(args):
        scaler = StandardScaler().fit(arg)

    return scaler

In [ ]:
def get_images(df):
    images = []

    for idx, row in df.iterrows():
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)

        band_1_scaler = standard_scale(band_1)
        band_2_scaler = standard_scale(band_2)

        band_1 = band_1_scaler.transform(band_1)
        band_2 = band_2_scaler.transform(band_2)

        bands = np.dstack((band_1, band_2))
        images.append(bands)

    return np.array(images)

# Load 

In [ ]:
PROJ_ROOT = os.path.join(os.path.abspath(os.path.dirname(__file__)), '..', '..')
train_df = pd.read_json(os.path.join(PROJ_ROOT, 'data', 'train.json'), dtype='float32')
test_df = pd.read_json(os.path.join(PROJ_ROOT, 'data', 'test.json'),  dtype='float32')

In [ ]:
# Training parameters.
batch_size = 32
num_classes = 2
epochs = 5

# Embedding dimensions.
row_hidden = 128
col_hidden = 128